In [1]:
from smarty.forcefield import *
import parmed
import openeye.oechem as oechem
from simtk.openmm.app import PDBFile

In [2]:
pdbfile = PDBFile('cyclohexane_ethanol_0.4_0.6.pdb')
ff = ForceField('../../smarty/data/forcefield/Frosst_AlkEtOH.ffxml')

mols = []
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
for molname in ['cyclohexane', 'ethanol']:
    ifs = oechem.oemolistream(molname+'.mol2')
    ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
    mol = oechem.OEGraphMol()
    while oechem.OEReadMolecule(ifs, mol):
        oechem.OETriposAtomNames(mol)
        mols.append(oechem.OEGraphMol(mol))

system = ff.createSystem( pdbfile.topology, mols)
#Swap in positions
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdbfile.positions)

# Create parmed Structure
structure = parmed.openmm.topsystem.load_topology( pdbfile.topology, system, pdbfile.positions)
# Write AMBER parameter/crd
#parmed.amber.AmberParm( system, structure)
#help(structure.save)
#Save prmtop
structure.save('system.prmtop', overwrite=True)
# Save crd
structure.save('system.crd', format='rst7', overwrite=True)

In [1]:
# Evaluate energy of OpenMM system and AMBER system to check that they are still the same
from smarty.forcefield_utils import *
#ambertop, ambersys, amberpos = create_system_from_amber( 'system.prmtop', 'system.crd')
#groups0, groups1, energy0, energy1 = compare_system_energies( ambertop, pdbfile.topology, ambersys, system)
#print(energy0, energy1)

# Well, that's not good -- it fails on creating the AMBER system. Is it an issue with the AMBER files? 
# Let's cross compare with those in the distribution
# This runs properly, while the above does not. So it's some issue with the AMBER files.
ambertop, ambersys, amberpos = create_system_from_amber( '../../../smarty/data/systems/amber/cyclohexane_ethanol_0.4_0.6.prmtop', '../../../smarty/data/systems/amber/cyclohexane_ethanol_0.4_0.6.inpcrd')

In [47]:
dir(system)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__swig_getmethods__',
 '__swig_setmethods__',
 '__weakref__',
 'addConstraint',
 'addForce',
 'addParticle',
 'getConstraintParameters',
 'getDefaultPeriodicBoxVectors',
 'getForce',
 'getForces',
 'getNumConstraints',
 'getNumForces',
 'getNumParticles',
 'getParticleMass',
 'getVirtualSite',
 'isVirtualSite',
 'removeConstraint',
 'removeForce',
 'setConstraintParameters',
 'setDefaultPeriodicBoxVectors',
 'setParticleMass',
 'setVirtualSite',
 'this',
 'usesPeriodicBoundaryConditions']

In [44]:
# Seems to have zero charges for subsequent instances of molecule in topology; is this correct?

# Troubleshoot charge issue - print charges as seen in parmed
dir(structure.residues[2].atoms[0])
structure.residues[1].atoms[0].charge
for atom in structure.residues[0]: print atom.charge
for atom in structure.residues[1]: print atom.charge
    
# Compare with charges in oemols - these are correct
print("Charges in oemols:")
for mol in mols:
    for atom in mol.GetAtoms():
        print atom.GetPartialCharge()
        
# Can I access charges in OpenMM system?

-0.0966999977827
0.129700005054
-0.599500000477
0.0447999984026
0.0447999984026
0.0447999984026
0.017100000754
0.017100000754
0.397899985313
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
Charges in oemols:
-0.076099999249
-0.0754999965429
-0.0754999965429
-0.0754999965429
-0.0754999965429
-0.0754999965429
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
0.0377999991179
-0.0966999977827
0.129700005054
-0.599500000477
0.0447999984026
0.0447999984026
0.0447999984026
0.017100000754
0.017100000754
0.397899985313
